In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
from malaya_speech.train.model import demucs, stft
import tensorflow as tf
import numpy as np


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
i = tf.placeholder(tf.float32, [None, 1])
model = demucs.Model(i, sources = 1, logging = True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Tensor("Pad_1:0", shape=(?, 99668, 1), dtype=float32)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Tensor("sequential/conv1d_1/mul:0", shape=(?, 24916, 64), dtype=float32)
Tensor("sequential_2/conv1d_4/mul:0", shape=(?, 6228, 128), dtype=float32)
Tensor("sequential_4/conv1d_7/mul:0", shape=(?, 1556, 256), dtype=float32)
Tensor("sequential_6/conv1d_10/mul:0", shape=(?, 388, 512), dtype=float32)
Tensor("sequential_8/conv1d_13/mul:0", shape=(?, 96, 1024), dtype=float32)
x 

In [5]:
logits = model.logits[:tf.shape(i)[0]]

In [6]:
# https://github.com/facebookresearch/denoiser/blob/master/denoiser/stft_loss.py#L109
stft_loss = stft.loss.TFMultiResolutionSTFT(factor_sc = 0.1, factor_mag = 0.1)

In [7]:
l1 = tf.reduce_mean(tf.abs(i - logits))

In [8]:
# https://github.com/facebookresearch/denoiser/blob/master/denoiser/solver.py
sc_loss, mag_loss = stft_loss(tf.expand_dims(i[:,0], 0), tf.expand_dims(logits[:,0], 0))
loss = l1 + sc_loss + mag_loss

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [10]:
y, sr = malaya_speech.load('../speech/7021-79759-0004.flac', sr = 44100)
y = y[:sr * 10]
len(y), len(y) / sr

(441000, 10.0)

In [11]:
%%time

r = sess.run(model.logits, feed_dict = {i: np.expand_dims(y, -1)})

CPU times: user 20.7 s, sys: 8.81 s, total: 29.6 s
Wall time: 5.32 s


In [12]:
r.shape

(443740, 1)

In [13]:
%%time

sess.run(loss, feed_dict = {i: np.expand_dims(y, -1)})

CPU times: user 22.2 s, sys: 8.31 s, total: 30.5 s
Wall time: 5.53 s


array([0.46085644], dtype=float32)